In [413]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, csv, string, re

def cleanup(text):
    punctuation = '[!-.:?;@/…|⠀_=♥️"]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_s = re.sub(r"http\S+", "", text_subbed)
    ttt = re.sub(r'''((?:https?://|www\d{0,3}[.]|[a-z0-9.-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|(([^\s()<>]+|(([^\s()<>]+)))))+(?:(([^\s()<>]+|(([^\s()<>]+))))|[^\s`!()[]{};:'".,<>?«»“”‘’]))''', " ", text_s)
    return ttt

In [414]:
#Classificando como categoria #Níveis da categoria
dados = pd.read_excel('t1.xlsx')
dados.loc[:,'relevância'] = dados['relevância'].astype('category')
dados.relevância.cat.categories = ['Irrelevante', 'Relevante']
dados.head()

,num,Treinamento,relevância
0,0,nunca vi um mercado tão cheio,Irrelevante
1,1,rt @folha: guedes diz que vai acionar stf cont...,Relevante
2,2,deixando tudo mais barato:\ncupom ifood (app) ...,Irrelevante
3,3,rt @reporterlacerda: inovação no grenal\n\npre...,Irrelevante
4,4,@ericat_lol mercado a vista vai ficar parado p...,Relevante


In [416]:
dados_r = dados[dados.relevância == 'Relevante']
l_ttr = dados_r.Treinamento.tolist()
#pra cada palavra na lista do tt, vai criar uma lista só com todas as palavras
l_ttr_split = []
for i in l_ttr:
    l_ttr_split.append(i.split())

output=open('prt_rel.txt','w',encoding="utf-8-sig")
#lista com palavras limpas
lpal_r = []
for lista in l_ttr_split:
    for pal in lista:
        c_pal = cleanup(pal)
        if c_pal != "":
            lpal_r.append(c_pal)

s_relev = pd.Series(lpal_r)
t_relev = s_relev.value_counts(True)

In [417]:
dados_i = dados[dados.relevância == 'Irrelevante']

#tt em lista 
l_tti = dados_i.Treinamento.tolist()

#pra cada palavra na lista do tt, vai criar uma lista só com todas as palavras
l_tti_split = []
for i in l_tti:
    l_tti_split.append(i.split())

output=open('prt_i.txt','w',encoding="utf-8-sig")
#lista com palavras limpas
lpal_i = []
for lista in l_tti_split:
    for pal in lista:
        c_pal = cleanup(pal)
        if c_pal != "":
            lpal_i.append(c_pal)
            
s_ire = pd.Series(lpal_i)
t_ire = s_ire.value_counts(True)

In [427]:
for i in dados.num:
    lpal_i_temp = lpal_i
    lpal_r_temp = lpal_r
    tt = dados.iloc[i,1] #pegou o tt inteiro
    tt_split = tt.split() #splitou o tt, agora é uma lista com cada elemento sendo uma palavra
    frase = []
    for pal in tt_split:
        frase.append(cleanup(pal)) #cada elemento de frase é uma palavra do tweet já limpa
        
    #smoothing
    for plvr in frase:
        if plvr not in lpal_i_temp:
            lpal_i_temp.append(plvr)
        if plvr not in lpal_r_temp:
            lpal_r_temp.append(plvr)

    #atualiza séries e tabela
    s_relev_temp = pd.Series(lpal_r_temp)
    t_relev_temp = s_relev_temp.value_counts(True)
    
    s_ire_temp = pd.Series(lpal_i_temp)
    t_ire_temp = s_ire_temp.value_counts(True)
    
    #total palavras
    portug = lpal_i_temp + lpal_r_temp
    s_prtg_temp = pd.Series(portug)
    
    #calcula probabilidade pelo numero de palavras
    p_relev = len(s_relev_temp)/len(s_prtg_temp)
    p_irrelev = len(s_ire_temp)/len(s_prtg_temp)

    try:
        pf_dado_r = t_relev_temp[frase].prod()
        pf_dado_i = t_ire_temp[frase].prod()

        pr_dado_f = pf_dado_r * p_relev
        pi_dado_f = pf_dado_i * p_irrelev
        
        if pr_dado_f > pi_dado_f:
            if (pr_dado_f > pi_dado_f):
                dados.loc[dados.num==i,'Classificador'] = 'Relevante'
        else:
            dados.loc[dados.num==i,'Classificador'] = 'Irrelevante'     
            
    except:
        print("erro frase ")
        print(frase)
        print('')



In [428]:
dados

,num,Treinamento,relevância,Classificador
0,0,nunca vi um mercado tão cheio,Irrelevante,Irrelevante
1,1,rt @folha: guedes diz que vai acionar stf cont...,Relevante,Relevante
2,2,deixando tudo mais barato:\ncupom ifood (app) ...,Irrelevante,Irrelevante
3,3,rt @reporterlacerda: inovação no grenal\n\npre...,Irrelevante,Irrelevante
4,4,@ericat_lol mercado a vista vai ficar parado p...,Relevante,Relevante
...,...,...,...,...
295,295,"@santosgxbx a expectativa é essa, porém na rea...",Irrelevante,Irrelevante
296,296,rt @folha_mercado: guedes diz que vai acionar ...,Relevante,Relevante
297,297,"dólar sobe, o brasileiro médio imediatamente u...",Relevante,Relevante
298,298,rt @vinisantucci: fui ao mercado comprar café\...,Irrelevante,Irrelevante
